# Notebook to test functions while the communication is not working

Libraries:
- pycryptodome : pip install pycryptodome
- cryptography : pip install cryptography
- pmlb: pip install pmlb
- pandas : pip install pandas

In [15]:
import pandas as pd
from io import StringIO
from Crypto.Cipher import AES
from Crypto.Util import Counter
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
import json
import os
from cryptography.hazmat.primitives.serialization import Encoding
from cryptography.hazmat.primitives.serialization import PublicFormat
from cryptography.hazmat.primitives.serialization import load_pem_public_key
import base64

In [18]:
%pip install simplejson

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 74 kB 1.3 MB/s eta 0:00:01
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [19]:

file_path = os.path.join('dataset', 'infringement_dataset.csv')

#### Generate Private Key

In [20]:
def get_private_key():
    return get_random_bytes(AES.key_size[0])

    """
    with open('/dev/urandom', 'rb') as f:
        return f.read(AES.key_size[0])
    """

## Communication function

In [21]:
class CommunicationViaJson():
    def __init__(self):
        # Initializing the class.
        self.receive_information()

    def send_information(self):
        # Writing the data to a file.
        json_object = json.dumps(self.data, indent=4)
        with open('common_file.json', 'w') as output:
            output.write(json_object)
            output.close()
    
    def receive_information(self):
        # Reading the json file and loading it into the data variable.
        with open('common_file.json', 'r') as input:
            self.data = json.load(input)
            input.close()

    
    def add_value(self, variable, new_value):
        # A function that changes the value of a variable in the data and writes it into a json file
        self.data[variable] = new_value
        self.send_information()

    def get_value(self, variable):
        if variable in self.data:
            return self.data[variable]
        else:
            return None

    def get_hash_algorithm(self):
        pass
        # TODO: function to get the algorithm to use

## DH - generate private shared key

In [28]:
class DH_controlER:
    def __init__(self, communication):
        self.communication = communication
        self.parameters = dh.generate_parameters(generator=2, key_size=2048)
        self.private_key = self.parameters.generate_private_key()
        self.public_key = self.private_key.public_key()
        self.communication.add_value('P', self.parameters.parameter_numbers().p)
        self.communication.add_value('G', self.parameters.parameter_numbers().g)

        with open('controlKey', 'wb') as file:
            file.write(self.public_key.public_bytes(encoding=Encoding.PEM, format=PublicFormat.SubjectPublicKeyInfo)) 
            file.close()
        #self.communication.add_value('public_key_controlER', self.public_key.public_bytes(encoding=Encoding.PEM, format=PublicFormat.SubjectPublicKeyInfo))
        
    def calculate_private_key(self):
        with open('delenttureKey', 'rb') as f:
            decoded = f.read()
            f.close()
        #decoded = self.communication.get_value('public_key_delentture')
        other_public_key = load_pem_public_key(decoded)
        algorithm = hashes.SHA256() # TODO: change algorithm
        if other_public_key:
            self.shared_secret = self.private_key.exchange(other_public_key)
            self.private_key = HKDF(
                algorithm = algorithm, 
                length=32,
                salt=None, 
                info=b'handshake data',
                backend=default_backend()
            ).derive(self.shared_secret)
            
        return None

In [30]:
class DH_delentture:
    def __init__(self, communication):
        self.communication = communication
        p = self.communication.get_value('P')
        g = self.communication.get_value('G')
        self.pn = dh.DHParameterNumbers(p, g)
        self.parameters = self.pn.parameters()
        self.private_key = self.parameters.generate_private_key()
        self.public_key = self.private_key.public_key()
        with open('delenttureKey', 'wb') as file:
            file.write(self.public_key.public_bytes(encoding=Encoding.PEM, format=PublicFormat.SubjectPublicKeyInfo)) 
            file.close()
        #self.communication.add_value('public_key_delentture', self.public_key.public_bytes(encoding=Encoding.PEM, format=PublicFormat.SubjectPublicKeyInfo))
        
    def calculate_private_key(self):
        with open('controlKey', 'rb') as f:
            decoded = f.read()
            f.close()
        #decoded = self.communication.get_value('public_key_controlER')
        other_public_key = load_pem_public_key(decoded)
        algorithm = hashes.SHA256() # TODO: change algorithm
        if other_public_key:
            self.shared_secret = self.private_key.exchange(other_public_key)
            self.private_key = HKDF(
                algorithm = algorithm, 
                length=32,
                salt=None,
                info=b'handshake data',
                backend=default_backend()
            ).derive(self.shared_secret)
            
        return None
    
    

### Testing

In [23]:
communication = CommunicationViaJson()

FileNotFoundError: [Errno 2] No such file or directory: 'common_file.json'

In [164]:
dh_control_ER = DH_controlER(communication)

In [165]:
communication2 = CommunicationViaJson()

In [166]:
dh_delentture = DH_delentture(communication2)

In [29]:
control_private = dh_control_ER.calculate_private_key()

NameError: name 'dh_control_ER' is not defined

In [168]:
delentture_private = dh_delentture.calculate_private_key()

## AES-CTR

### Encrypt

In [27]:
def encrypt_csv_ctr(file_name, civ, counter_size=128):
    key = get_private_key()
    
    counter = Counter.new(counter_size, initial_value=civ)
    
    aes = AES.new(key, mode=AES.MODE_CTR, counter=counter)

    with open(file_name, 'rb') as file:
        csv_data = file.read()
        encrypted_csv = aes.encrypt(csv_data)
    return encrypted_csv, key

ctr_iv = 10
ctr_ct, ctr_key = encrypt_csv_ctr(file_path, ctr_iv)

FileNotFoundError: [Errno 2] No such file or directory: 'dataset/infringement_dataset.csv'

### Decrypt

In [170]:
def decrypt_csv_ctr(ct, key, counter_iv, counter_size=128):
    counter = Counter.new(counter_size, initial_value=counter_iv)    
    aes = AES.new(key=key, mode=AES.MODE_CTR, counter=counter)
    return aes.decrypt(ct)

In [171]:
pd.read_csv(StringIO(decrypt_csv_ctr(ctr_ct, ctr_key, ctr_iv).decode('utf-8'))).head(3)

,loan_id,infringed,contract_type,gender,has_own_car,has_own_realty,num_children,annual_income,credit_amount,credit_annuity,...,SK_ID_CURR,avg_days_decision,past_avg_amount_annuity,past_avg_amt_application,past_avg_amt_credit,past_loans_approved,past_loans_refused,past_loans_canceled,past_loans_unused,past_loans_total
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,100002.0,606.0,9251.775,179055.0,179055.0,1.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,100003.0,1305.0,56553.990,435436.5,484191.0,3.0,0.0,0.0,0.0,3.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,100004.0,815.0,5357.250,24282.0,20106.0,1.0,0.0,0.0,0.0,1.0


### Verification 

In [31]:
def signature(data):
    #The hashing algorithm used to hash the data is SHA256, and the padding algorithm is pS%
    padding_obj_sig = padding.PSS(mgf=padding.MGF1(hashes.SHA256()), salt_length=padding.PSS.MAX_LENGTH)
    signature = control_private.sign(data, padding=padding_obj_sig, algorithm=hashes.SHA256()) 
    return signature,padding_obj_sig

#Function used to verify the integraty of the data, teh receiver must use the same hash and padding algorithm
def integraty(data,padding_obj_sig,public_key):
    #If the signature does not match the data, an InvalidSignature exception will be raised
    decrypted_file = public_key.verify(signature,data,padding_obj_sig,hashes.SHA256())

